In [ ]:
from skimage import io, transform
import glob
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import math
from tensorflow.python import keras
from keras import Sequential, Model
from keras.layers import InputLayer, Input, Reshape, MaxPooling2D, Conv2D, Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras import optimizers
from keras import applications

In [ ]:
IMG_SIZE = 56
IMG_SHAPE = (IMG_SIZE, IMG_SIZE)
NUM_CLASS = 5
NUM_CHANNELS = 3
IMG_SHAPE_FULL = (IMG_SIZE, IMG_SIZE, NUM_CHANNELS)

In [ ]:
def read_imgs(path):
    euro_dirs = [path + '/' + x for x in os.listdir(path) if os.path.isdir(path + '/' + x)]
    imgs = list()
    labels = list()
    for idx, folder in enumerate(euro_dirs):
        for filename in glob.glob(folder + '/*.jp*g'):
            img = io.imread(filename)
            img = transform.resize(img, IMG_SHAPE)
            
            imgs.append(img)
            labels.append(idx)
            
    return np.asarray(imgs, np.float32), np.asarray(labels, np.int32)

imgs, cls = read_imgs('./data/coins')

In [ ]:
labels = np.zeros((cls.size, 5))
labels[np.arange(cls.size), cls] = 1

indices = np.arange(imgs.shape[0])
np.random.shuffle(indices)
imgs = imgs[indices]
labels = labels[indices]

ratio = 0.8
s = int(imgs.shape[0] * ratio)
x_train = imgs[:s]
y_train = labels[:s]
x_test = imgs[s:]
y_test = labels[s:]

In [ ]:
# Start construction of the Keras Sequential model.
model = Sequential()

model.add(InputLayer(input_shape=IMG_SHAPE_FULL))

model.add(Conv2D(kernel_size=5, strides=1, filters=4, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))

model.add(Conv2D(kernel_size=5, strides=1, filters=4, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2))

model.add(Flatten())
model.add(Dense(5, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x=x_train,
          y=y_train,
          validation_split=0.2,
          epochs=20, batch_size=512)

In [ ]:
result = model.evaluate(x=x_test, y=y_test, batch_size=512)
for name, value in zip(model.metrics_names, result):
    print(name, value)

In [ ]:
session = backend.get_session()
constant_graph = graph_util.convert_variables_to_constants(session, session.graph_def, [out.op.name for out in model.outputs])

with tf.gfile.FastGFile('./data/model.pb', mode='wb') as f:
    f.write(constant_graph.SerializeToString()),
    
tflite_model = tf.contrib.lite.toco_convert(constant_graph, model.inputs, model.outputs),
with open('../CoinCounter/Data/model2.tflite', 'wb') as f:
    f.write(tflite_model)
